In [29]:
# -*- coding: utf-8 -*- 

import re 
from tqdm import tqdm 
import time
from datetime import datetime
import sqlite3
import sys
import os
import pandas as pd
import unify

In [8]:
def connect(file_path, primary, columns):
    con = sqlite3.connect(file_path)
    cur = con.cursor()
    cols = ", ".join([c + ' Varchar' for c in columns]) 
    cur.execute("create table meta ("+primary+" Varchar PRIMARY KEY, "+cols+" )")
    cur.execute("CREATE INDEX log on meta (textid);")
    cur.execute("create table plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL);")
    cur.execute("create table tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL );")
    con.commit()
    return con, cur

In [36]:
workdir = r'/home/tari/Загрузки/taiga/nplus1'
filename = 'nplus1.db'
file_path = os.path.join(workdir, filename)
metatablepath = os.path.join(workdir,'newmetadata.csv')
tagged = os.path.join(workdir,'texts_tagged')
plain = os.path.join(workdir,'texts')

meta = pd.read_csv(metatablepath, sep='\t', encoding='utf8')
meta = meta.fillna('')
meta.head()

,segment,textid,textname,textregion,textrubric,textdiff,author,authortexts,authorreaders,magazine,date,time,tags,source
0,nplus1,20151029radar,Французы испытали пассивный «летающий радар»,,Оружие,2.1,Василий Сычев,,,,29 Окт. 2015,16:47,,https://nplus1.ru/news/2015/10/29/radar
1,nplus1,20161212aggressive-drones,Для агрессивных маневров дронам хватило камеры...,,Технологии,2.6,Николай Воронцов,,,,12 Дек. 2016,17:27,,https://nplus1.ru/news/2016/12/12/aggressive-d...
2,nplus1,20150826twinjet-hubble,«Хаббл» получил новые фотографии «крыльев» Баб...,,Технологии,1.1,,,,,26 Авг. 2015,17:46,,https://nplus1.ru/news/2015/08/26/twinjet-hubble
3,nplus1,20150826shining-solved-profile,Криминологи научились вычислять будущих бытовы...,,,4.4,,,,,26 Авг. 2015,14:15,,https://nplus1.ru/news/2015/08/26/shining-solv...
4,nplus1,20160111snow,Ford начал снежные испытания беспилотного авто...,,Транспорт,1.7,Василий Сычев,,,,11 Янв. 2016,09:44,,https://nplus1.ru/news/2016/01/11/snow


In [40]:
if not os.path.exists(filename):
    con, cur = connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
else:
    con = sqlite3.connect(filename, meta.columns[1], [meta.columns[0]]+list(meta.columns[2:]))
    cur = con.cursor()

In [32]:
cur.execute("SELECT * FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

[('table', 'meta', 'meta', 2, 'CREATE TABLE meta (textid Varchar PRIMARY KEY, segment Varchar, textname Varchar, textregion Varchar, textrubric Varchar, textdiff Varchar, author Varchar, authortexts Varchar, authorreaders Varchar, magazine Varchar, date Varchar, time Varchar, tags Varchar, source Varchar )'), ('table', 'plain_texts', 'plain_texts', 5, 'CREATE TABLE plain_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL)'), ('table', 'tagged_texts', 'tagged_texts', 7, 'CREATE TABLE tagged_texts (id Varchar(128) NOT NULL PRIMARY KEY, text Varchar NOT NULL )')]


In [21]:
meta.iloc[6].to_dict()

{'author': 'Николай Воронцов',
 'authorreaders': '',
 'authortexts': '',
 'date': '07 Окт. 2015',
 'magazine': '',
 'segment': 'nplus1',
 'source': 'https://nplus1.ru/news/2015/10/07/makerarm-adam',
 'tags': '',
 'textdiff': 1.6000000000000001,
 'textid': '20151007makerarm-adam',
 'textname': 'На Kickstarter появилось два проекта 3D-МФУ',
 'textregion': '',
 'textrubric': 'Гаджеты',
 'time': '16:00'}

In [ ]:
for i in range(len(meta)):
    values = meta.iloc[i].to_dict()
    values['textid'] = str(values['textid'])
    values['textdiff'] = str(values['textdiff'])
    columns = ', '.join(values.keys())
    
    #print(list(values.values()))
    placeholders = ', '.join('?' * len(values))
    sql = 'INSERT INTO meta ({}) VALUES ({})'.format(columns, placeholders)
    #print(sql)
    cur.execute(sql, list(values.values()))
    
    valuest = {'id': values['textid'], 'text': unify.open_text(os.path.join(plain, str(values['textid'])+".txt"))}
    
    columns = ', '.join(valuest.keys())
    placeholders = ', '.join('?' * len(valuest))
    sql2 = 'INSERT INTO plain_texts ({}) VALUES ({})'.format(columns, placeholders) 
    cur.execute(sql2, list(valuest.values()))
    try:
        valuest2 = {'id': values['textid'], 'text': unify.open_text(open(os.path.join(tagged, str(values['textid'])+".txt"),'r', encoding='utf8').read())}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    except:
        valuest2 = {'id': values['textid'], 'text': ""}
        columns = ', '.join(valuest2.keys())
        placeholders = ', '.join('?' * len(valuest2))
        sql3 = 'INSERT INTO tagged_texts ({}) VALUES ({})'.format(columns, placeholders) 
        cur.execute(sql3, list(valuest2.values()))
    con.commit()

In [ ]:
(textid Varchar PRIMARY KEY, segment Varchar, textname Varchar, textregion Varchar, textrubric Varchar, textdiff Varchar, author Varchar, authortexts Varchar, authorreaders Varchar, magazine Varchar, date Varchar, time Varchar, tags Varchar, source Varchar )